### Emotion models - Considering the division of each signal in 5 excerpts:  

##### Feature optimization, model training and evaluation and hyperparameters optimization for each dataset and for each combination between groups formed and set of features. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

from time import time 
import random
import itertools
from itertools import repeat

from scipy import stats
from scipy.spatial.distance import pdist, squareform 
from scipy.cluster.hierarchy import dendrogram, linkage
from statsmodels.stats.anova import AnovaRM

from sklearn.utils import shuffle
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error, classification_report, accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, ShuffleSplit, GroupShuffleSplit
from sklearn.feature_selection import SelectFromModel

In [2]:
# Train set composed of 75% of participants and test set composed by the reamining 25% of participants:
def train_test_diff_part_equal_emotion (df_, part_list, test_size):
    
    random.shuffle(part_list)
    
    n_train = round((1-test_size) * len(part_list))
    n_test = len(part_list) - n_train
    
    train_indices = random.sample(part_list, k=n_train)
    test_indices = []
    for part in part_list:
        if part not in train_indices:
            test_indices.append(part)
    
    train_set = df_[df_['Participante'].isin(train_indices)].copy()
    test_set = df_[df_['Participante'].isin(test_indices)].copy()
    
    return train_set, test_set

# Train set composed by all participants and emotions (Use of 4 excerpts per participant and emotion)
# And test set composed by all participants and emotions (Use of the remaining excerpt per participant and emotion):
def train_test_equal_part_emotion (data):
    
    e_val = []
    for part in data['Participante'].unique().tolist():
        for emotion in data['Labels'].unique().tolist():
            e = int(random.choice(data['Excerpt'].unique().tolist()))
            e_val.append(e)

    train_set=data.copy()
    test_set=pd.DataFrame(columns=data.columns)
    i=0
    for part in data['Participante'].unique().tolist():
        test_set = test_set.append(train_set[(train_set['Participante']==part) & (train_set['Labels']=='Fear') & (train_set['Excerpt']==e_val[i])])
        train_set.drop(train_set[(train_set['Participante']==part) & (train_set['Labels']=='Fear') & (train_set['Excerpt']==e_val[i])].index, inplace = True)
        i=i+1
        test_set = test_set.append(train_set[(train_set['Participante']==part) & (train_set['Labels']=='Happy') & (train_set['Excerpt']==e_val[i])])
        train_set.drop(train_set[(train_set['Participante']==part) & (train_set['Labels']=='Happy') & (train_set['Excerpt']==e_val[i])].index, inplace = True)
        i=i+1
        test_set = test_set.append(train_set[(train_set['Participante']==part) & (train_set['Labels']=='Neutral') & (train_set['Excerpt']==e_val[i])])
        train_set.drop(train_set[(train_set['Participante']==part) & (train_set['Labels']=='Neutral') & (train_set['Excerpt']==e_val[i])].index, inplace = True)
        i=i+1
    
    
    return train_set, test_set

#### Acessing and standardizating data:

In [3]:
def acess_data (filename, sheetname, signal):

    #Acessing data
    data = pd.read_excel(filename, sheet_name=sheetname)
        
    #Standardizating (Signals)
    if signal:
        feat = data.drop(['Labels','Participante'], axis=1).columns
        scaler = MinMaxScaler()
        data[feat] = scaler.fit_transform(data[feat])
    
    #Standardizating (Signal excerpts)
    else:
        feat = data.drop(['Labels','Participante', 'Excerpt'], axis=1).columns
        scaler = MinMaxScaler()
        data[feat] = scaler.fit_transform(data[feat])
        
    return data

#### Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)

In [4]:
# Evaluating the performance of the emotional model according to the number of features/variables/pcs (extra feature selection -> select the features/variables/pcs that are more important in the SVM model) )
def recursive_feat_elimination (x, y):
    
    # Recursive Elimination with cross-validation
    tic = time()
    svc = SVC(kernel="linear")
    min_feat_select = 1  # Minimum number of features/variables/pcs to consider
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(2), scoring='accuracy', min_features_to_select=min_feat_select)
    
    #StratifiedKFold for multiclass or binary y
    rfecv.fit(x, y)
    toc = time()
    print('Duration {:.2f} seconds'.format(toc - tic))

    optimal_number_feat = rfecv.n_features_
    print('\nOptimal number of variables: ',rfecv.n_features_) #Variables because it can be principal components or signal features
    optimal_selected_feat = x.columns[rfecv.support_].to_list()
    print('\nSelected variable(s): ', optimal_selected_feat)

    # Plot number of features VS. cross-validation scores
    plt.figure(figsize=(13,5))
    plt.xlabel("Number of variables selected")
    plt.ylabel("Cross validation score")
    range_feat = range(min_feat_select,len(rfecv.grid_scores_) + min_feat_select)
    cross_val_score=rfecv.grid_scores_
    plt.plot(range(min_feat_select,len(rfecv.grid_scores_) + min_feat_select), rfecv.grid_scores_)
    plt.grid()
    plt.show()
    
    return optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score

#### Emotional model:

In [5]:
def model_svm (xtrain, xtest, ytrain, ytest):

    # GridSearchCV
    parameters = [{'kernel':['rbf'], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001],'C':[0.1, 1, 10, 100, 1000]},
                  {'kernel': ['linear'], 'C':[0.1, 1, 10, 100, 1000]},
                  {'kernel': ['poly'], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001],'C':[0.1, 1, 10, 100, 1000],'degree':[0,1,2,3,4,5,6]}]
    classifier = SVC()
    model_GSCV = GridSearchCV(classifier, parameters, verbose=3, return_train_score=True) # scoring='accuracy',
    
    # Fitting
    model_GSCV.fit(xtrain,ytrain) #The model is using the best paramenters
    print('\nBest parameters set found on development set:')
    print(model_GSCV.best_params_)
    

    # Predicting
    GSCV_predictions_train = model_GSCV.predict(xtrain)
    GSCV_predictions_test = model_GSCV.predict(xtest)
    
    results_train = pd.DataFrame()
    results_test = pd.DataFrame()
    results_train['Truth'] = ytrain
    results_test['Truth'] = ytest
    results_train['Predicted Emotion Train'] = GSCV_predictions_train
    results_test['Predicted Emotion Test'] = GSCV_predictions_test
    
    # Classification report
    print('\nClassification Report Train:')
    train_rep = classification_report(ytrain,GSCV_predictions_train, digits=3)
    dic_train = classification_report(ytrain,GSCV_predictions_train, digits=3, output_dict=True)
    print(train_rep)
    print('\nClassification Report Test:')
    test_rep= classification_report(ytest,GSCV_predictions_test, digits=3)
    dic_test = classification_report(ytest,GSCV_predictions_test, digits=3, output_dict=True)
    print(test_rep)
    
    ac_train = accuracy_score(ytrain,GSCV_predictions_train)
    ac_test = accuracy_score(ytest,GSCV_predictions_test)
    
    print("\nAccuracy Train: ", ac_train)
    print("\nAccuracy Test:: ", ac_test)
    
    return model_GSCV.best_params_, results_train, results_test, dic_train, dic_test

#### Cross Validation:

In [6]:
def cross_validation (data, optimal_feat, split1, split2, split3, split4, test_size, X):
    
    dic = {}
    
    for i in range(0, X):
    
        dic[i]={}
        
        # Splitting data
        if split1==True:
            participants_list = data['Participante'].unique()
            train, test = train_test_equal_part_diff_emotion(data,participants_list)
            
        elif split2==True:
            participants_list = data['Participante'].unique().tolist()
            train, test = train_test_diff_part_equal_emotion (data, participants_list, test_size)
        
        elif split3==True:
            train, test = train_test_equal_part_emotion (data)
            
        elif split4==True:
            train, test = split_Fear_Happy (data)
        
        x_train = train[optimal_feat].values
        x_test = test[optimal_feat].values
        y_train = train['Labels'].values
        y_test = test['Labels'].values
        
        # Index of train and test
        index_train = train.index.tolist()
        index_test = test.index.tolist()

        # Emotional Model
        best_params, results_train, results_test, dic_train, dic_test = model_svm (x_train, x_test, y_train, y_test)
        
        print('\n')
        
        # May happen that the best hyper-parameter of kernel is linear so there is no gamma associated
        if 'gamma' not in best_params.keys():
            gamma = '-'
        else :
            gamma = best_params['gamma']
        
        if 'degree' not in best_params.keys():
            degree = '-'
        else :
            degree = best_params['degree']
        
        neutral_train = dic_train['Neutral']['f1-score']
        neutral_test = dic_test['Neutral']['f1-score']
        
        # Creating dictionary with information of each run
        dic[i] = {'C':best_params['C'],'gamma':gamma,'degree':degree,'kernel':best_params['kernel'], 
                  'AccuracyTrain':dic_train['accuracy'], 'AccuracyTest':dic_test['accuracy'], 
                  'FearTrain':dic_train['Fear']['f1-score'], 'FearTest':dic_test['Fear']['f1-score'], 
                  'HappyTrain':dic_train['Happy']['f1-score'], 'HappyTest':dic_test['Happy']['f1-score'], 
                  'NeutralTrain':neutral_train, 'NeutralTest':neutral_test,
                  'IndexTrain': index_train, 'IndexTest':index_test}

    # Printing Results
    print('----------------------------------------Final Results----------------------------------------')
    results = pd.DataFrame.from_dict(dic, orient='index')
    
    print('\nTrain Results:')
    print('\nAverage Accuracy: ', results.AccuracyTrain.mean())
    print('\nStandard Deviation Accuracy: ', results.AccuracyTrain.std())
    print('\nAverage Accuracy - Fear: ', results.FearTrain.mean())
    print('Average Accuracy - Happy: ', results.HappyTrain.mean())
    print('Average Accuracy - Neutral: ', results.NeutralTrain.mean())
    
    print('\nTest Results:')
    print('\nAverage Accuracy: ', results.AccuracyTest.mean())
    print('\nStandard Deviation Accuracy: ', results.AccuracyTest.std())
    print('Max Test Accuracy: ', results.AccuracyTest.max())
    print('Min Test Accuracy: ', results.AccuracyTest.min())
    print('\nAverage Accuracy - Fear: ', results.FearTest.mean())
    print('Average Accuracy - Happy: ', results.HappyTest.mean())
    print('Average Accuracy - Neutral: ', results.NeutralTest.mean())    
    
    return results

#### Train, validation and test:

In [7]:
run = 30 

Datasets (Considering the division of each signal in 5 excerpts): 

- Dataset 4: Dataset 4 makes use of the principal components obtained through principal component analysis on the selected features according to methodology A or B. 


- Dataset 5: Dataset 5 makes use of the features generated from the approach C2 (Seleção de features sobre a matriz completa(fear+happy+neutral):  Dataset_1_geral_feat_select_corr).


- Dataset 6: Dataset 6 makes use of the features generated from the approach C1 (Seleção de features separadamente e união das features em matriz final: Dataset_1_features). 

### Dataset 4: 

In [ ]:
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='Geral_EMG_EDA_ECG', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','PC11','PC12','PC13','PC14','PC15','PC16']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo Geral_EMG: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='Geral_EMG', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo Geral_ECG: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='Geral_ECG', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','PC11','PC12']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo Geral_EDA: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='Geral_EDA', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G1_EMG_ECG_EDA: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G1_EMG_EDA_ECG', signal=True)
selected_feat = ['PC1','PC2','PC3']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G1_EMG: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G1_EMG', signal=True)
selected_feat = ['PC1','PC2','PC3']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G1_ECG: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G1_ECG', signal=True)
selected_feat = ['PC1','PC2','PC3']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G1_EDA: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G1_EDA', signal=True)
selected_feat = ['PC1','PC2','PC3']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G2_EMG_ECG_EDA: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G2_EMG_EDA_ECG', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G2_EMG: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G2_EMG', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G2_ECG: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G2_ECG', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G2_EDA: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G2_EDA', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G3_EMG_ECG_EDA: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G3_EMG_EDA_ECG', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','PC11','PC12','PC13']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G3_EMG: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G3_EMG', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G3_ECG: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G3_ECG', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo G3_EDA: 
data = acess_data(filename='Dataset_2_pca_excerpts_2.xlsx',sheetname='G3_EDA', signal=True)
selected_feat = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8']
results_split2 = cross_validation (data, selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

### Dataset 5: 

In [ ]:
# Global group & ECG+EMG+EDA features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='Geral_ECG+EMG+EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G1 & ECG+EMG+EDA features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G1_ECG+EMG+EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G2 & ECG+EMG+EDA features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G2_ECG+EMG+EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G3 & ECG+EMG+EDA features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G3_ECG+EMG+EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo Geral_EMG: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='Geral_EMG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

print(results_split2)

In [ ]:
# Group G1 & EMG features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G1_EMG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G2 & EMG features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G2_EMG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G3 & EMG features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G3_EMG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo Geral_EDA: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='Geral_EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)
optimal_selected_feat= ['EDA_Tonic_Mean', 'SCR_Amplitude_Var','SCR_RiseTime_Skew']
results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

print(results_split2)

In [ ]:
# Group G1 & EDA features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G1_EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G2 & EDA features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G2_EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G3 & EDA features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G3_EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Grupo Geral_ECG: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='Geral_ECG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

print(results_split2)

In [ ]:
# Group G1 & ECG features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G1_ECG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G2 & ECG features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G2_ECG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G3 & ECG features: 
data = acess_data(filename='Dataset_2_geral_feat_select_corr.xlsx',sheetname='G3_ECG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

### Dataset 6: 

In [ ]:
# Global group & ECG+EMG+EDA features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='Geral_ECG+EMG+EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G1 & ECG+EMG+EDA features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G1_ECG+EMG+EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)
+
results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G2 & ECG+EMG+EDA features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G2_ECG+EMG+EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G3 & ECG+EMG+EDA features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G3_ECG+EMG+EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Global group & EMG features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='Geral_EMG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G1 & EMG features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G1_EMG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G2 & EMG features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G2_EMG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G3 & EMG features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G3_EMG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Global group & EDA features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='Geral_EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G1 & EDA features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G1_EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G2 & EDA features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G2_EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G3 & EDA features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G3_EDA', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Global group & ECG features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='Geral_ECG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G1 & ECG features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G1_ECG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

#optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G2 & ECG features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G2_ECG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)

In [ ]:
# Group G3 & ECG features: 
data = acess_data(filename='Dataset_2_features.xlsx',sheetname='G3_ECG', signal=True)

# Feature Selection with Recursive Feature Elimination with Cross-Validation (RFECV)
X = data.iloc[:,3:] 
Y = data['Labels'] 

optimal_number_feat, optimal_selected_feat, range_feat, cross_val_score = recursive_feat_elimination (X, Y)

#optimal_selected_feat = ['ECG_Rate_Mean','ECG_Rate_Skew','ECG_Rate_Kurt','ECG_Tduration_Mean','ECG_Tduration_Var','ECG_Tduration_Skew','ECG_Tduration_Kurt','HRV_TINN','HRV_MinNN','HRV_LFn','HRV_CSI_Modified','HRV_LF','HRV_PSS','HRV_PAS','HRV_GI','HRV_Ca','HRV_PI','HRV_MFDFA_alpha1_Peak', 'HRV_MFDFA_alpha1_Fluctuation','HRV_MFDFA_alpha1_Asymmetry','HRV_MFDFA_alpha1_Increment','HRV_MFDFA_alpha2_Width','HRV_MFDFA_alpha2_Peak','HRV_MFDFA_alpha2_Asymmetry','HRV_MFDFA_alpha2_Fluctuation','HRV_MFDFA_alpha2_Max','HRV_MFDFA_alpha2_Increment', 'HRV_RCMSEn','HRV_MSEn','HRV_KFD','HRV_CMSEn']                        

results_split2 = cross_validation (data, optimal_selected_feat, split1=False, split2=False, split3=True, split4=False,
                                   test_size=0.25, X=run)